In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def rk4_step(state, f, dt, *args):
    '''
    state: a tuple that include all nodes of each variable.
    f: the specific expression of the equation, the expression of the time derivative.
    dt: time step
    '''

    # k1
    k1 = f(state, *args)
    # add dt/2*k1 to each component of state
    state_k1 = tuple(s + 0.5 * dt * k for s, k in zip(state, k1))

    # k2
    k2 = f(state_k1, *args)
    # add dt/2*k2 to each component of state
    state_k2 = tuple(s + 0.5 * dt * k for s, k in zip(state, k2))

    # k3
    k3 = f(state_k2, *args)
    # add dt * k3 to each component of state
    state_k3 = tuple(s + dt* k for s, k in zip(state, k3))

    # k4
    k4 = f(state_k3, *args)

    # update
    new_state = tuple(
        s + (dt/6.0) * (k1_i + 2*k2_i + 2*k3_i + k4_i)
        for s, k1_i, k2_i, k3_i, k4_i in zip(state, k1, k2, k3, k4)
    )

    return new_state

def wave_equation(state, c, dx):
    u, v = state
    N = len(u) - 1

    du_dt = np.zeros_like(u)
    dv_dt = np.zeros_like(v)

    for i in range(1, N):
        du_dt[i] = v[i]
        dv_dt[i] = c**2 * (u[i+1] - 2*u[i] + u[i-1]) / dx**2

    # Fixed boundary condition
    # du_dt[0] = 0.0
    # dv_dt[0] = 0.0
    # du_dt[N] = 0.0
    # dv_dt[N] = 0.0

    # Periodic boundary condition
    du_dt[0] = v[0]
    dv_dt[0] = c**2 * (u[1] - 2*u[0] + u[N]) / dx**2

    du_dt[N] = v[N]
    dv_dt[N] = c**2 * (u[0] - 2*u[N] + u[N-1]) / dx**2

    return (du_dt, dv_dt)

def coupled_wave_equations(state, c1, c2, a1, a2, b1, dx):
    u1, v1, u2, v2 = state
    N = len(u1) - 1

    du1_dt = np.zeros_like(u1)
    du2_dt = np.zeros_like(u1)
    dv1_dt = np.zeros_like(u2)
    dv2_dt = np.zeros_like(u2)

    for i in range(1, N):
        du1_dt[i] = v1[i]
        # dv1_dt[i] = (c1**2 * (u1[i+1] - 2*u1[i] + u1[i-1]) / dx**2) + a1 * (u2[i+1] - u2[i-1]) / (2 * dx)
        dv1_dt[i] = (c1**2 * (u1[i+1] - 2*u1[i] + u1[i-1]) / dx**2) + a1 * (u2[i+1] - u2[i-1]) / (2 * dx) + b1*u2[i]**2

        du2_dt[i] = v2[i]
        dv2_dt[i] = (c2**2 * (u2[i+1] - 2*u2[i] + u2[i-1]) / dx**2) + a2 * (u1[i+1] - u1[i-1]) / (2 * dx)
    
    du1_dt[0] = 0
    dv1_dt[0] = 0
    du2_dt[0] = 0
    dv2_dt[0] = 0
    du1_dt[N] = 0
    dv1_dt[N] = 0
    du2_dt[N] = 0
    dv2_dt[N] = 0

    return (du1_dt, dv1_dt, du2_dt, dv2_dt)

def coupled_soliton_wave_equations(state, K, theta0, alpha, dx):
    u, v, theta, phi = state    # phi is the time derivative of theta
    N = len(u) - 1

    du_dt = np.zeros_like(u)
    dv_dt = np.zeros_like(u)
    dtheta_dt = np.zeros_like(theta)
    dphi_dt = np.zeros_like(phi)

    for i in range(1, N):
        du_dt[i] = v[i]
        dv_dt[i] = (u[i+1] - 2*u[i] + u[i-1]) / dx**2 + (1 - K) * np.tan(theta0) * (theta[i+1] - theta[i-1]) / (2*dx)
        dtheta_dt[i] = phi[i]
        dphi_dt[i] = alpha**2 * ((np.cos(2 * theta0) - K) * (theta[i+1] - 2*theta[i] + theta[i-1]) / dx**2 
                               - 2 * np.sin(2 * theta0) * (u[i+1] - u[i-1]) / (2*dx) 
                               - 4 * (2*K + np.cos(theta0)**2 * (u[i+1] - u[i-1]) / (2 * dx) + 2 * np.sin(theta0)**2) * theta[i] 
                               - 4 * np.sin(2 * theta0) * theta[i]**2)
        
    # No boundary condition
    du_dt[0] = v[0]
    dv_dt[0] = (u[1] - u[0]) / dx**2 + (1 - K) * np.tan(theta0) * (theta[1] - theta[0]) / (2*dx)
    dtheta_dt[0] = phi[0]
    dphi_dt[0] = alpha**2 * ((np.cos(2 * theta0) - K) * (theta[1] - theta[0]) / dx**2 
                               - 2 * np.sin(2 * theta0) * (u[1] - u[0]) / (2*dx) 
                               - 4 * (2*K + np.cos(theta0)**2 * (u[1] - u[0]) / (2 * dx) + 2 * np.sin(theta0)**2) * theta[0] 
                               - 4 * np.sin(2 * theta0) * theta[0]**2)
    du_dt[N] = v[N]
    dv_dt[N] = (-u[N] + u[N-1]) / dx**2 + (1 - K) * np.tan(theta0) * (theta[N] - theta[N-1]) / (2*dx)
    dtheta_dt[N] = phi[N]
    dphi_dt[N] = alpha**2 * ((np.cos(2 * theta0) - K) * (-theta[N] + theta[N-1]) / dx**2 
                               - 2 * np.sin(2 * theta0) * (u[N] - u[N-1]) / (2*dx) 
                               - 4 * (2*K + np.cos(theta0)**2 * (u[N] - u[N-1]) / (2 * dx) + 2 * np.sin(theta0)**2) * theta[N] 
                               - 4 * np.sin(2 * theta0) * theta[N]**2)
    
    return (du_dt, dv_dt, dtheta_dt, dphi_dt)

In [ ]:
def square_rhombus_coupled_wave_equations(state, m1, m2, I1, I2, k1, k2, k, k_theta, theta0, l, dx):
    # u, v, theta are the disp, w, z, phi are the velocity
    u11, w11, u21, w21, u12, w12, u22, w22, v11, z11, v21, z21, v12, z12, v22, z22, theta11, phi11, theta21, phi21, theta12, phi12, theta22, phi22 = state
    N = len(u11) - 1

    du11_dt = np.zeros_like(u11)
    dw11_dt = np.zeros_like(w11)
    du21_dt = np.zeros_like(u21)
    dw21_dt = np.zeros_like(w21)
    du12_dt = np.zeros_like(u12)
    dw12_dt = np.zeros_like(w12)
    du22_dt = np.zeros_like(u22)
    dw22_dt = np.zeros_like(w22)
    dv11_dt = np.zeros_like(v11)
    dz11_dt = np.zeros_like(z11)
    dv21_dt = np.zeros_like(v21)
    dz21_dt = np.zeros_like(z21)
    dv12_dt = np.zeros_like(v12)
    dz12_dt = np.zeros_like(z12)
    dv22_dt = np.zeros_like(v22)
    dz22_dt = np.zeros_like(z22)
    dtheta11_dt = np.zeros_like(theta11)
    dphi11_dt = np.zeros_like(phi11)
    dtheta21_dt = np.zeros_like(theta21)
    dphi21_dt = np.zeros_like(phi21)
    dtheta12_dt = np.zeros_like(theta12)
    dphi12_dt = np.zeros_like(phi12)
    dtheta22_dt = np.zeros_like(theta22)
    dphi22_dt = np.zeros_like(phi22)

    for i in range(1, N):
        du11_dt[i] = w11[i]
        dw11_dt[i] = k / m1 * (u11[i+1] - 2*u11[i] + u11[i-1]) / dx**2  \
                     + (2 * k_theta * theta11[i] * np.cos(theta0) + 2 * (k_theta + k * l**2) * np.sin(theta0)) / l / m1 * (theta11[i+1] - theta11[i-1]) / (2*dx) \
                     + (-2 * k * l * u12[i] + k * l * u21[i] - k_theta * theta21[i] * np.cos(theta0) + k * l**2 * theta21[i] * np.cos(theta0) + k_theta * theta11[i] * theta21[i] * np.sin(theta0)) / l / m1
        du21_dt[i] = w21[i]
        dw21_dt[i] = k / m1 * (u21[i+1] - 2*u21[i] + u21[i-1]) / dx**2  \
                     + (-2 * k_theta * theta21[i] * np.cos(theta0) + 2 * (-k_theta + k * l**2) * np.sin(theta0)) / l / m1 * (theta21[i+1] - theta21[i-1]) / (2*dx) \
                     + (k * l * (u11[i] + u21[i] - 2 * u22[i]) + k_theta * (theta11[i] - theta21[i]) * np.cos(theta0) + k * l**2 * (theta11[i] - theta21[i]) * np.cos(theta0) - k_theta * theta11[i] * theta21[i] * np.sin(theta0)) / l / m1
        dv11_dt[i] = z11[i]
        dz11_dt[i] = k / m1 * (v11[i+1] - 2*v11[i] + v11[i-1]) / dx**2  \
                     + (2 * (k_theta - k * l**2) * np.cos(theta0) - 2 * k_theta * theta11[i] * np.sin(theta0)) / l / m1 * (theta11[i+1] - theta11[i-1]) / (2*dx) \
                     + (-2 * k * l * v12[i] + k * l * v21[i] + k_theta * theta11[i] * theta21[i] * np.cos(theta0) + k_theta * theta21[i] * np.sin(theta0) + k * l**2 * theta21[i] * np.sin(theta0)) / l / m1
        dv21_dt[i] = z21[i]
        dz21_dt[i] = k / m1 * (v21[i+1] - 2*v21[i] + v21[i-1]) / dx**2  \
                     + (2 * (k_theta + k * l**2) * np.cos(theta0) - 2 * k_theta * theta21[i] * np.sin(theta0)) / l / m1 * (theta21[i+1] - theta21[i-1]) / (2*dx) \
                     + (k * l * v11[i] - k * l * v21[i] - 2 * k * l * v22[i] + k_theta * theta11[i] * theta21[i] * np.cos(theta0) + k_theta * theta11[i] * np.sin[theta0] - k_theta * theta21[i] * np.sin(theta0) + k * l**2 * (-theta11[i] + theta21[i]) * np.sin[theta0]) / l / m1
        dtheta11_dt[i] = phi11[i]
        dphi11_dt[i] = - (k_theta - k * l**2 * np.cos(2 * theta0) + k * l**2 * theta11[i] * np.sin(2 * theta0)) / I1 * (theta11[i+1] - 2*theta11[i] + theta11[i-1]) / dx**2 \
                       + (2 * k * l * theta21[i] * np.cos(theta0) + 2 * k * l * np.sin(theta0)) / I1 * (u21[i+1] - u21[i-1]) / (2*dx) \
                       + (-2 * k * l * np.cos(theta0) + 2 * k * l * theta21[i] * np.sin(theta0)) / I1 * (v21[i+1] - v21[i-1]) / (2*dx) \
                       + (4 * k * l**2 * theta21[i] 
                          - k_theta * (theta11[i] + 9 * theta21[i] - 2 * theta22[i]) 
                          + k * l * np.cos(theta0) * (-u11[i] + u21[i] - v11[i] * theta21[i] - v21[i] * theta21[i])
                          + k * l**2 * np.cos(2 * theta0) * (-theta11[i] - 5 * theta21[i] + 2 * theta22[i])
                          + k * l * np.sin(theta0) * (-v11[i] - v21[i] - u11[i] * theta21[i] - u21[i] * theta21[i])
                          + k * l**2 * np.sin(2 * theta0) * (theta11[i] * theta21[i] + 4 * theta21[i]**2 - 2 * theta21[i] * theta22[i])) / I1




SyntaxError: invalid syntax (3281144164.py, line 2)